In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210417.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice]
0,13/04/2021 17:43:41,The Beyond (1981),Hard Times (1975),McCabe & Mrs. Miller (1971),YOUR FAVORITE DIRECTOR'S FAVORITE DIRECTOR,I FUCKING HATE SCIENCE,ETHICAL INCELS,[Default to Nicolas Cage]
1,13/04/2021 17:46:09,McCabe & Mrs. Miller (1971),Hard Times (1975),The Beyond (1981),YOUR FAVORITE DIRECTOR'S FAVORITE DIRECTOR,ETHICAL INCELS,I FUCKING HATE SCIENCE,[Default to Nicolas Cage]
2,13/04/2021 17:55:57,Hard Times (1975),NaN,NaN,ETHICAL INCELS,NaN,NaN,NaN
3,13/04/2021 19:02:47,McCabe & Mrs. Miller (1971),Hard Times (1975),The Beyond (1981),I FUCKING HATE SCIENCE,YOUR FAVORITE DIRECTOR'S FAVORITE DIRECTOR,[Default to Nicolas Cage],NaN
4,15/04/2021 09:04:30,Hard Times (1975),McCabe & Mrs. Miller (1971),The Beyond (1981),[Default to Nicolas Cage],YOUR FAVORITE DIRECTOR'S FAVORITE DIRECTOR,I FUCKING HATE SCIENCE,ETHICAL INCELS
5,15/04/2021 15:13:28,The Beyond (1981),Hard Times (1975),McCabe & Mrs. Miller (1971),ETHICAL INCELS,I FUCKING HATE SCIENCE,YOUR FAVORITE DIRECTOR'S FAVORITE DIRECTOR,[Default to Nicolas Cage]


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['The Beyond (1981)', 'Hard Times (1975)',
       'McCabe & Mrs. Miller (1971)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'The Beyond (1981)': <Candidate('The Beyond (1981)')>,
 'Hard Times (1975)': <Candidate('Hard Times (1975)')>,
 'McCabe & Mrs. Miller (1971)': <Candidate('McCabe & Mrs. Miller (1971)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(The Beyond (1981), Hard Times (1975), McCabe & Mrs. Miller (1971))>,
 <Ballot(McCabe & Mrs. Miller (1971), Hard Times (1975), The Beyond (1981))>,
 <Ballot(Hard Times (1975))>,
 <Ballot(McCabe & Mrs. Miller (1971), Hard Times (1975), The Beyond (1981))>,
 <Ballot(Hard Times (1975), McCabe & Mrs. Miller (1971), The Beyond (1981))>,
 <Ballot(The Beyond (1981), Hard Times (1975), McCabe & Mrs. Miller (1971))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                      Votes  Status
---------------------------  -------  --------
Hard Times (1975)                  2  Hopeful
McCabe & Mrs. Miller (1971)        2  Hopeful
The Beyond (1981)                  2  Rejected

FINAL RESULT
Candidate                      Votes  Status
---------------------------  -------  --------
Hard Times (1975)                  4  Elected
McCabe & Mrs. Miller (1971)        2  Rejected
The Beyond (1981)                  0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                     Votes  Status
------------------------------------------  -------  --------
YOUR FAVORITE DIRECTOR'S FAVORITE DIRECTOR        2  Hopeful
ETHICAL INCELS                                    2  Hopeful
I FUCKING HATE SCIENCE                            1  Rejected
[Default to Nicolas Cage]                         1  Rejected

FINAL RESULT
Candidate                                     Votes  Status
------------------------------------------  -------  --------
YOUR FAVORITE DIRECTOR'S FAVORITE DIRECTOR        4  Elected
ETHICAL INCELS                                    2  Rejected
I FUCKING HATE SCIENCE                            0  Rejected
[Default to Nicolas Cage]                         0  Rejected

